In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
import math


In [12]:
def create_discriminator():
    input_shape = (None, None, 3)  # Input shape for the discriminator
    inputs = layers.Input(shape=input_shape)
    x = layers.Conv2D(64, 3, strides=1, padding='same')(inputs)
    x = layers.LeakyReLU(alpha=0.2)(x)

    x = layers.Conv2D(64, 3, strides=2, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU(alpha=0.2)(x)

    x = layers.Conv2D(128, 3, strides=1, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU(alpha=0.2)(x)

    x = layers.Conv2D(128, 3, strides=2, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU(alpha=0.2)(x)

    x = layers.Conv2D(256, 3, strides=1, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU(alpha=0.2)(x)

    x = layers.Conv2D(256, 3, strides=2, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU(alpha=0.2)(x)

    x = layers.Flatten()(x)
    x = layers.Dense(1024)(x)
    x = layers.LeakyReLU(alpha=0.2)(x)
    x = layers.Dense(1, activation='sigmoid')(x)

    model = Model(inputs, x, name='discriminator')
    return model


In [21]:
def create_generator():
    inputs = layers.Input(shape=(None, None, 3))
    x = inputs

    # Encoder
    skips = []
    for i in range(16):
        channels = 2 ** (i + 4)
        x = layers.Conv2D(channels, 3, strides=1, padding='same')(x)
        x = layers.LeakyReLU(alpha=0.2)(x)
        if i < 8:
            skips.append(x)
        x = layers.Conv2D(channels, 3, strides=2, padding='same')(x)
        x = layers.LeakyReLU(alpha=0.2)(x)

    # Decoder
    for i in reversed(range(16)):
        channels = 2 ** (i + 4)
        x = layers.Conv2DTranspose(channels, 3, strides=2, padding='same')(x)
        x = layers.LeakyReLU(alpha=0.2)(x)
        if i < 8:
            skip = skips.pop()
            x = layers.Concatenate()([x, skip])
        x = layers.Conv2DTranspose(channels, 3, strides=1, padding='same')(x)
        x = layers.LeakyReLU(alpha=0.2)(x)
    outputs = layers.Conv2DTranspose(3, 3, strides=1, padding='same', activation='sigmoid')(x)
    model = Model(inputs, outputs, name='generator')
    return model


In [22]:
def create_gan(generator, discriminator):
    discriminator.trainable = False
    gan_input = layers.Input(shape=(None, None, 3))
    fake_output = generator(gan_input)
    gan_output = discriminator(fake_output)
    gan_model = Model(gan_input, gan_output, name='gan')
    return gan_model


In [23]:
dataset_url = 'https://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_train_LR_bicubic_X4.zip'
data_dir = 'C:/Users/Admin/.keras/datasets/div2k'
batch_size = 8
epochs = 10

# Prepare dataset
image_size = 128
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    label_mode=None,
    batch_size=batch_size,
    image_size=(image_size, image_size),
    interpolation='bicubic',
    seed=1337
)


Found 800 files belonging to 1 classes.


In [24]:
train_ds = train_ds.map(lambda x: x / 255.0)


In [25]:
generator = create_generator()
discriminator = create_discriminator()
gan = create_gan(generator, discriminator)


ResourceExhaustedError: {{function_node __wrapped__AddV2_device_/job:localhost/replica:0/task:0/device:CPU:0}} OOM when allocating tensor with shape[3,3,4096,4096] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator mklcpu [Op:AddV2]

In [ ]:
def image_preprocessing(image):
    gray_image = []
    for row in image:
        gray_row = []
        for pixel in row:
            gray_value = int(0.2989 * pixel[0] + 0.5870 * pixel[1] + 0.1140 * pixel[2])
            gray_row.append(gray_value)
        gray_image.append(gray_row)

    contrast_image = []
    for row in gray_image:
        contrast_row = []
        for pixel in row:
            contrast_value = int(1.5 * (pixel - 128) + 128)
            contrast_row.append(contrast_value)
        contrast_image.append(contrast_row)

    rotated_image = []
    for i in range(len(contrast_image)):
        for j in range(len(contrast_image[0])):
            x = i * math.cos(math.pi / 4) - j * math.sin(math.pi / 4)
            y = i * math.sin(math.pi / 4) + j * math.cos(math.pi / 4)
            if 0 <= x < len(contrast_image) and 0 <= y < len(contrast_image[0]):
                rotated_pixel = contrast_image[int(x)][int(y)]
                rotated_image.append(rotated_pixel)
    shifted_image = []
    for i in range(len(rotated_image)):
        for j in range(len(rotated_image[0])):
            x = i - 50
            y = j - 50
            if 0 <= x < len(rotated_image) and 0 <= y < len(rotated_image[0]):
                shifted_pixel = rotated_image[int(x)][int(y)]
                shifted_image.append(shifted_pixel)

    return shifted_image